# Effects of taVNS on HRV 
## Discription
2 groups (true label temporally unknown, each patient received VNS for several days during which there were 2 sessions of stimulation). Subject ID from 2020004 to 2020015.

**Warning**: Both the number of days and the number of sessions depend on the subject.
## Objective
- To see the Effects of taVNS on HRV
- Unsupervised clustering to see whether we can distinguise the two groups.

## Input
- nn_interval file (.pickle) for each subject
- info csv (stimulation_timestamped.csv) indicating when the stimulation begins 
**Warning**: the info file was automatically generated. It needs a second check.

## Output
- For each group (patient), we explore the effect of VNS on HRV (boxplot, pre/during/post-stim)
- Taking day as a variable, two independent variable: day and session
- clustering including Features engineering (e.g. $HRV_{during} - \frac{HRV_{pre} + HRV_{post}}{2})$ per patient) and unsupervised learning (e.g. kmeans, DBSCAN)

## Workflow
### 1. double check df_info
- the length of the stimulation should be 20 min
- there should be two stimulations per patient in a single day

## data loading

In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import datetime

data_dir = os.path.expanduser("~/Desktop/GT/ECG_VNS/data")
# os.path.exists(data_dir)  # test

info_filename = os.path.join(data_dir, 'stimulation_timestamped.csv')
df_info = pd.read_csv(info_filename, index_col=0)

## filter bad rows in df_info (drop rows)
### Untackled problem:
- in this version, data were discarded if multipul plausible stimulation onsets were found
### improvement 2 implement:
- loops for list creation and concatenation

In [2]:
# raw by raw scanning
# criteria: 20 min duration
# timestamp format: 2/22/21 20:18
tolerence = 100 # 100 seconds
VNS_duration = 1200 # 1200 seconds

list_subj2filtered_df = []
list_date2filtered_df = []
list_time_s2filtered_df = []
list_switch2filtered_df = []  # 1 stands for on, 0 stands for off

array_subjs = df_info['subject'].unique()
for subj in array_subjs:
    df_per_subj = df_info[df_info['subject'] == subj]
    list_date = []
    list_time_s = []
    for index, row in df_per_subj.iterrows():
        # month/day/year
#         date = row['timestamp'].split('/')[0] + '/' + row['timestamp'].split('/')[1] + row['timestamp'].split('/')[2]
        date = row['timestamp'].split(' ')[0] 
        time = row['timestamp'].split('/')[2].split(' ')[1]
        hour = int(time.split(':')[0])
        minute = int(time.split(':')[1])
        time_s = datetime.timedelta(hours=hour, minutes=minute).total_seconds()
        list_date.append(date)
        list_time_s.append(time_s)
    list_time_s = np.array(list_time_s)
    list_date = np.array(list_date)
    _, unique_indices = np.unique(list_time_s, return_index=True)
    unique_indices = np.sort(unique_indices)
    # make sure that there are either 2 or 4 stim each day
    list_time_s = list_time_s[unique_indices]
    list_date = list_date[unique_indices]
    
    list_time_s_updated = []
    list_date_updated = []
    for date in np.unique(list_date):
        indices2search = np.where(list_date == date)[0]
        find_stim_onset = False
        num_stim_found = 0
        for i in indices2search:
            for j in indices2search[np.where(indices2search == i)[0][0] + 1:]:
                if (VNS_duration - tolerence) < (list_time_s[j] - list_time_s[i]) < (VNS_duration + tolerence):
                    list_time_s_updated.append(list_time_s[i])
                    list_time_s_updated.append(list_time_s[j])
                    find_stim_onset = True
                    num_stim_found +=1
        
                        
        if num_stim_found > 2:  # more than two stim onsets were found
#             list_date_updated = list_date_updated[list_date_updated != date]
            list_time_s_updated = list_time_s_updated[:-2 * num_stim_found]
        else:
            for i in range(num_stim_found):
                list_date_updated.append(date)
    list_subj2filtered_df = list_subj2filtered_df + [subj] * len(list_time_s_updated)
    list_date2filtered_df = list_date2filtered_df + [date for date in list_date_updated for _ in (0, 1)]
    list_time_s2filtered_df = list_time_s2filtered_df + list_time_s_updated
    list_switch2filtered_df = list_switch2filtered_df + [i for j in range(len(list_date_updated)) for i in [1, 0] ]

# we create new df from arrays here
df_info_filtered = pd.DataFrame(data=np.array([np.array(list_subj2filtered_df), np.array(list_date2filtered_df),
                                               np.array(list_time_s2filtered_df), 
                                      np.array(list_switch2filtered_df)]).T,
                                    columns=['subj', 'date', 'time_s', 'switch'])
df_info_filtered.to_csv(os.path.join(data_dir, 'stimulation_timestamped_filtered.csv'))
# Test properties here via assert. This applies when 

In [3]:
# Let's see how it looks like
# load the csv if necessary
df_info_filtered.head(12)

,subj,date,time_s,switch
0,2020004,2/22/21,73080.0,1
1,2020004,2/22/21,74340.0,0
2,2020004,2/23/21,75780.0,1
3,2020004,2/23/21,77040.0,0
4,2020004,2/24/21,27300.0,1
5,2020004,2/24/21,28560.0,0
6,2020004,2/24/21,71700.0,1
7,2020004,2/24/21,72960.0,0
8,2020004,2/25/21,31800.0,1
9,2020004,2/25/21,33060.0,0


## extract pre/during/post-stm from pickle files
Output format:

| subj  | date | time |    session      | nn_interval|
| ----- | ---- | ---- | --------------- | ---------- |
|2020014| 2/23 |am/pm | pre/during/post |    value   |

**Be careful, this might take 8h on mac**

In [81]:
noon_s = 12*60*60
def am_or_pm(time_s):
    if 0 < time_s < noon_s:
        output = 'am'
    else:
        output = 'pm'
    return output

def datetime2time_s(datetime_series):
    '''
    datetime is a pd series, no vectorized operation is possible
    '''
    time_s = []
    for i in datetime_series:
        hours = i.hour
        minutes = i.minute
        seconds = i.second
        time_s.append(datetime.timedelta(hours=hours, minutes=minutes, 
                                seconds=seconds).total_seconds())
    return time_s

subj2df = []
date2df = []
time2df = []
session2df = []
nn_interval2df = []
for subj_id in array_subjs:
    nn_filename = os.path.join(data_dir, str(subj_id) + '.pickle')
    df = pd.read_pickle(nn_filename)
#     # truncate to speed up
#     df = df.truncate(before=175000, after= 176000)
#     # reindex by time_s
#     df['time_s_idx'] = datetime2time_s(df['timestamp'])
#     df.set_index('time_s_idx')
    df_info_per_subj = df_info_filtered[df_info_filtered['subj'] == str(subj_id)]
    dates = pd.unique(df_info_per_subj['date'])
    df_info_per_subj_day = df_info_per_subj[df_info_per_subj['date'] == date]
    
    for index, row in df.iterrows():
        # date conversion
        date = row['timestamp'].strftime('%-m/%d/%y')
        time_s = datetime.timedelta(hours=row['timestamp'].hour, minutes=row['timestamp'].minute, 
                            seconds = row['timestamp'].second).total_seconds()
        
        df_info_per_subj_day = df_info_per_subj[df_info_per_subj['date'] == date]
        onset_pre_VNS = df_info_per_subj_day[df_info_per_subj_day['switch'] == '1'].reset_index()['time_s'].astype(float)\
        - VNS_duration
        offset_pre_VNS = df_info_per_subj_day[df_info_per_subj_day['switch'] == '1'].reset_index()['time_s'].astype(float)

        onset_post_VNS = df_info_per_subj_day[df_info_per_subj_day['switch'] == '0'].reset_index()['time_s'].astype(float)
        offset_post_VNS = df_info_per_subj_day[df_info_per_subj_day['switch'] == '0'].reset_index()['time_s'].astype(float)\
        + VNS_duration
        time = am_or_pm(time_s)
        
        for i in range(int(df_info_per_subj_day.shape[0]/2)):  # we have one stim in the morning, one in the afternoon
            if onset_pre_VNS[i] < time_s < offset_pre_VNS[i]:
                date2df.append(date)
                session2df.append('pre')
                time2df.append(time)
                nn_interval2df.append(row['nn_interval'])
                subj2df.append(subj_id)
            elif offset_pre_VNS[i] < time_s < onset_post_VNS[i]:
                date2df.append(date)
                session2df.append('during')
                time2df.append(time)
                nn_interval2df.append(row['nn_interval'])
                subj2df.append(subj_id)
            elif onset_post_VNS[i] < time_s < offset_post_VNS[i]:
                date2df.append(date)
                session2df.append('post')
                time2df.append(time)
                nn_interval2df.append(row['nn_interval'])
                subj2df.append(subj_id)
            else:
                continue
df_filtered = pd.DataFrame(data=np.array([np.array(subj2df), np.array(date2df),np.array(time2df), np.array(session2df),
                                          np.array(nn_interval2df)]).T,
                                    columns=['subj', 'date', 'time', 'session', 'nn_interval'])
df_filtered.to_csv(os.path.join(data_dir, 'nn_interval_whole.csv'))                

In [69]:
# onset_post_VNS[i]
df_info_per_subj_day
df_info_per_subj_day[df_info_per_subj_day['switch'] == '0'].reset_index()['time_s']
for subj_id in array_subjs[0:1]:
    nn_filename = os.path.join(data_dir, str(subj_id) + '.pickle')
    df = pd.read_pickle(nn_filename)
    # truncate to speed up
    df = df.truncate(before=170000, after= 171000)
    # reindex by time_s
    df['time_s_idx'] = datetime2time_s(df['timestamp'])
    df.set_index('time_s_idx')

In [82]:
df_filtered.shape

(258505, 5)

In [64]:
df['timestamp'].to_numpy()[0]

numpy.datetime64('2021-02-24T07:13:25.960342000')